In [11]:
import numpy as np
import gvar as gv
import matplotlib.pyplot as plt
import math
import lsqfit
from scipy.fft import fft, fftfreq

In [2]:
def dft(points):
    ret = []
    for i in range(0,len(points)):
        tmpre = 0.
        tmpim = 0.
        for j in range(0,len(points)):
            tmpre += points[j]*gv.cos(-2.0*math.pi*i*j/len(points))
            tmpim += points[j]*gv.sin(-2.0*math.pi*i*j/len(points))
        ret.append([tmpre,tmpim])
        
    return ret

In [3]:
data_file = open("m_1_ns_2/finalreadoutdynamicdecouplingandrescaling_nairobi.csv")
data = np.loadtxt(data_file)
data_file.close()
data_gvar = gv.gvar(data[0],data[1:])

In [7]:
data_fftfreq = fftfreq(len(data[0]),1.0)
data_dft = dft(data_gvar)
data_dft_abs2 = [i[0]**2. + i[1]**2. for i in data_dft]
data_dft_abs2_means = [i.mean for i in data_dft_abs2]
data_dft_abs2_stdev = [i.sdev for i in data_dft_abs2]

In [9]:
plt.errorbar(data_fftfreq,data_dft_abs2_means,yerr=data_dft_abs2_stdev,fmt="x")

<ErrorbarContainer object of 3 artists>

In [20]:
y = {'data1' : data_dft_abs2}
x = {'data1' : data_fftfreq}

prior = {}
prior['a'] = gv.gvar(0.15,0.1)
prior['b'] = gv.gvar(1.0,1.0)
prior['c'] = gv.gvar(100.0,100.0)

def testfun(x, p):
    ans = {}
    for k in ['data1']:
        ans[k] =  p['c']/math.pi*(p['b']/((x[k]+p['a'])**2.+p['b']**2) + p['b']/((x[k]-p['a'])**2.+p['b']**2))
    return ans

fit = lsqfit.nonlinear_fit(data=(x, y), prior=prior, fcn=testfun, debug=True)
print(fit.format(maxline=True))       # print standard summary of fit

Least Square Fit:
  chi2/dof [dof] = 2.7e+02 [20]    Q = 0    logGBF = -2647.6

Parameters:
              a    -0.05 (10)     [  0.15 (10) ]  *
              b   9(7671)e-08     [  1.0 (1.0) ]  
              c       3 (100)     [  100 (100) ]  

Fit:
        key         y[key]         f(p)[key]
--------------------------------------------
    data1 0      3.5 (1.0)    0.00006 (5381)  ***
          1      0.49 (26)        1.426 (60)  ***
          2      8.8 (1.2)    0.00003 (2913)  *****
          3    228.6 (6.8)      10(8251)e-06  *****
          4     20.6 (2.0)       5(3998)e-06  *****
          5      3.00 (65)       3(2391)e-06  ****
          6      0.54 (31)       2(1601)e-06  *
          7      1.10 (45)       1(1151)e-06  **
          8      1.52 (53)        1(868)e-06  **
          9      1.15 (40)       8(6795)e-07  **
         10      1.41 (65)       6(5466)e-07  **
         11      1.15 (40)       8(6795)e-07  **
         12      1.52 (53)        1(868)e-06  **
         